<img src="files/model_1_outline.png">

# change log
### changing sequence rate 20 -> 5
### mobilenet

sequence generator shape (corrected) (b, n, w, h, c)

In [21]:
import tensorflow as tf
import datetime
import os
import glob
import keras

In [22]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [23]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [24]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [25]:
classes = ['CalotTriangleDissection',
 'CleaningCoagulation',
 'ClippingCutting',
 'GallbladderDissection',
 'GallbladderPackaging',
 'GallbladderRetraction',
 'Preparation']


classes.sort()
print(classes)

['CalotTriangleDissection', 'CleaningCoagulation', 'ClippingCutting', 'GallbladderDissection', 'GallbladderPackaging', 'GallbladderRetraction', 'Preparation']


In [26]:
# some global params
SIZE = (128, 128) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [27]:
# pattern to get videos and classes
glob_pattern='../cholec80/model_data/{classname}/*.avi'


In [28]:
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [29]:
# getting validation data
valid = train.get_validation_generator()

Total data: 7 classes for 1190 files for validation


In [30]:
import keras_video.utils
#keras_video.utils.show_sample(train)

In [31]:
#valid.files

## BUILD CONV NET

In [32]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
from keras.applications.mobilenet import MobileNet
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
        
    # adding a max pool
    output = GlobalMaxPool2D()

    return keras.Sequential([model, output])

## Build GRU

In [33]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [34]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 128, 128, 3)
print(INSHAPE)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

#this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

(5, 128, 128, 3)


## epochs, call backs

In [35]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210611-104208


In [36]:
EPOCHS=400
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/400
303/303 [==============================] - 69s 217ms/step - loss: 2.0398 - acc: 0.1404 - val_loss: 1.9458 - val_acc: 0.1436

Epoch 00001: saving model to model2_3_1_chkp\weights.01-1.95.hdf5
Epoch 2/400
303/303 [==============================] - 7s 24ms/step - loss: 1.9551 - acc: 0.1580 - val_loss: 1.9461 - val_acc: 0.1436

Epoch 00002: saving model to model2_3_1_chkp\weights.02-1.95.hdf5
Epoch 3/400
303/303 [==============================] - 7s 24ms/step - loss: 1.9505 - acc: 0.1218 - val_loss: 1.9458 - val_acc: 0.1360

Epoch 00003: saving model to model2_3_1_chkp\weights.03-1.95.hdf5
Epoch 4/400
303/303 [==============================] - 7s 24ms/step - loss: 1.9526 - acc: 0.1289 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00004: saving model to model2_3_1_chkp\weights.04-1.95.hdf5
Epoch 5/400
303/303 [==============================] - 7s 24ms/step - loss: 1.9488 - acc: 0.1326 - val_loss: 1.9460 - val_acc: 0.1436

Epoch 00005: saving model to model2_3_1_chkp\weights.05-1.9

KeyboardInterrupt: 